In [2]:
import pandas as pd
import gensim
from gensim.models import Word2Vec

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
import re

In [4]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rudi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
data_loc = 'data/'

In [7]:
data1 = pd.read_csv(data_loc + 'winemag-data_first150k.csv', index_col=0)
data2 = pd.read_csv(data_loc + 'winemag-data-130k-v2.csv', index_col=0)

In [8]:
data2 = data2.iloc[:75000]

In [9]:
#Panen andmed kokku
dataset = pd.concat([data1,data2])

In [10]:
dataset.reset_index(drop=True)

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225925,Austria,"There is a slight, aromatic lift on the nose t...",NaN,88,14.0,Kremstal,NaN,NaN,Grüner Veltliner,Winzer Krems,Anne Krebiehl MW,@AnneInVino,Winzer Krems 2015 Grüner Veltliner (Kremstal)
225926,New Zealand,"This plump, dry-style Pinot Gris features plen...",NaN,88,15.0,Marlborough,NaN,NaN,Pinot Gris,Yealands,Joe Czerwinski,@JoeCz,Yealands 2016 Pinot Gris (Marlborough)
225927,Italy,"Made with 80% Sangiovese and 20% Syrah, this h...",NaN,88,22.0,Tuscany,Toscana,NaN,Red Blend,Ziobaffa,Kerin O’Keefe,@kerinokeefe,Ziobaffa 2015 Red (Toscana)
225928,US,One of the first bottles to bear the name of t...,Single Vineyard,88,30.0,Idaho,NaN,NaN,Syrah,3 Horse Ranch Vineyards,Sean P. Sullivan,@wawinereport,3 Horse Ranch Vineyards 2014 Single Vineyard S...


In [11]:
#Vaatan, mis toimub
dataset.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN


In [12]:
desc = dataset.description

In [13]:
reviews_lower = [x.lower() for x in desc]

In [14]:
reviews_lower[0]

'this tremendous 100% varietal wine hails from oakville and was aged over three years in oak. juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. enjoy 2022–2030.'

In [15]:
#Puhastan teksti sodist, aga võite vaielda, et äkki ikkagi on vaja mingit tüüpi arve säilitada (nt aastaarvud)
cleaned_reviews = [re.sub('[^a-zA-Z]', ' ', x) for x in reviews_lower]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [16]:
cleaned_reviews[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy '

In [17]:
reviews = [nltk.sent_tokenize(x) for x in cleaned_reviews]

In [18]:
reviews = [y for x in reviews for y in x]

In [19]:
reviews[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy'

In [34]:
words = [nltk.word_tokenize(x) for x in reviews]

In [35]:
stop_words = stopwords.words('english')

In [36]:
for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(w) for w in words[i] if w not in stop_words]
    # words[i] = [w for w in words[i] if w not in stop_words]
# words = [[w for w in words[i] if w not in stop_words] for i in range(len(words))]

In [37]:
#Põhimõtteliselt on korpus valmis
words[0]

['tremendous',
 'varietal',
 'wine',
 'hail',
 'oakville',
 'aged',
 'three',
 'year',
 'oak',
 'juicy',
 'red',
 'cherry',
 'fruit',
 'compelling',
 'hint',
 'caramel',
 'greet',
 'palate',
 'framed',
 'elegant',
 'fine',
 'tannin',
 'subtle',
 'minty',
 'tone',
 'background',
 'balanced',
 'rewarding',
 'start',
 'finish',
 'year',
 'ahead',
 'develop',
 'nuance',
 'enjoy']

In [40]:
# Mudel, aga epohhe võiks rohkem olla (100)
model = gensim.models.Word2Vec(words, vector_size = 200 , window = 4 , min_count = 1,
                               seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)

In [41]:
model.save("word2vec.model")

In [42]:
#Kümme kõige lähedasemat sõna sõnale "magus"
model.wv.most_similar('sweet')

[('sweetened', 0.563584566116333),
 ('flavor', 0.4734434187412262),
 ('sweetness', 0.47125306725502014),
 ('jam', 0.44153374433517456),
 ('sugared', 0.4393143355846405),
 ('candied', 0.430242657661438),
 ('sugary', 0.4274977743625641),
 ('semisweet', 0.4259647727012634),
 ('vanilla', 0.4228847324848175),
 ('ripe', 0.42164573073387146)]

In [43]:
#Kümme kõige lähedasemat sõna sõnale "kuiv"
model.wv.most_similar('dry')

[('dryness', 0.49679720401763916),
 ('drily', 0.49370646476745605),
 ('crackingly', 0.3998042047023773),
 ('brisk', 0.37159591913223267),
 ('scour', 0.3637041449546814),
 ('semidry', 0.3478386700153351),
 ('bite', 0.3414926826953888),
 ('dryish', 0.34103474020957947),
 ('puckery', 0.33975422382354736),
 ('briskly', 0.33582451939582825)]

Päris asjani on tükk maad veel. Mu arust võiks jõuda selleni, et saaks veinisoovitusi nii karakterit kirjeldavate märksõnade kui veininime järgi (ehk mis veinid oleksid konkreetsele veinile kõige sarnasemad). Word2veci saab ära kasutada selles mõttes, et märksõna järgi otsides ei pea istuma me kinni ühe kindla sõna küljes (nt kui otsime kuiva veini, siis me ei saa vastu ainult neid, mille kirjelduses on konkreetselt sõna "kuiv"), vaid saame võtta arvesse ka selle sünonüüme/semantiliselt lähedasi sõnu. Siinse katsetuse põhjal saame ehk algatada arutelu?